# SDR Codebook optimization

## Setup

In [1]:
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import json

from sionna.rt import antenna_pattern

from cissir.physics import pow2db, db2power, mag2db

from cissir import params, optimization as opt
from cissir.beamforming import dft_codebook, plot_beamforming_polar
from cissir.raytracing import si_paths2cir, load_si_paths, normalize_si_taps
from cissir.utils import PrintBuffer, plot_path, res_path
from cissir import visualization as viz
viz.paper_style(line_cycler=False, figsize=(1.5 * viz.def_figsize[0], 1.5 * viz.def_figsize[1]))


In [2]:
# Global parameters

c = params.c
fc = params.fc_hz
wavelength_m = params.wavelength_m

ula_el_spacing = params.array_electrical_spacing

N_r = params.n_rx
N_t = params.n_tx

# Number of beams
L_r = params.num_beams
L_t = params.num_beams

### DFT beam codebook


In [3]:
dft_tx_codebook, tx_degs = dft_codebook(L_max = L_t , N1 = N_t, full_grid=True,
                                          az_min = -60, az_max = 60, transmit=True)
L_t = len(tx_degs)
dft_rx_codebook, rx_degs = dft_codebook(L_max = L_r , N1 = N_r, full_grid=True,
                                          az_min = -60, az_max = 60, transmit=False)
L_r = len(rx_degs)

In [4]:
si_num_taps = "full"   # Options: 1, 2, "full"

def cb_si_db(tx_codebook, rx_codebook, si_channel):
    return mag2db(opt.codebook_si(tx_codebook, rx_codebook, si_channel))

ht_si_full, _ = load_si_paths(num_taps="full")
h_full = si_paths2cir(ht_si_full)

if si_num_taps == "full":
    h_si = h_full
    si_db_ref = cb_si_db(dft_tx_codebook, dft_rx_codebook, h_si).max()
else:
    h_si, _ = load_si_paths(num_taps=si_num_taps)
    h_si, si_mag = normalize_si_taps(h_si, ht_si_full,
                                     dft_tx_codebook, dft_rx_codebook)
    si_db_ref = mag2db(si_mag)
    
    assert h_si.shape == (si_num_taps, N_r, N_t), "Unexpected SI shape"

In [5]:
h_si_rx, h_si_tx = opt.si_split(h_si)

min_eigval_db = mag2db(opt.min_eigval(h_si))
max_eigval_db = mag2db(opt.max_eigval(h_si))

min_split_db = pow2db(opt.min_eigval(h_si_rx)*opt.min_eigval(h_si_tx))
max_split_db = pow2db(opt.max_eigval(h_si_rx)*opt.max_eigval(h_si_tx))

In [6]:
print(f"      eig_min H: {min_eigval_db:.1f} dB")
print(f"eig_min Grx\Gtx: {min_split_db:.1f} dB")
print(f"            cHw: {si_db_ref:.1f} dB")
print(f"      eig_max H: {max_eigval_db:.1f} dB")
print(f"eig_max Grx\Gtx: {max_split_db:.1f} dB")

      eig_min H: -295.5 dB
eig_min Grx\Gtx: -155.1 dB
            cHw: -59.4 dB
      eig_max H: -59.8 dB
eig_max Grx\Gtx: -41.1 dB


## Optimization

In [7]:
feasible_tolerance = 1e-2
tgt_si_values = np.arange(si_db_ref, min_split_db, -1.0)
pbuf = PrintBuffer(print_input=False)

In [ ]:
rx_mesh, tx_mesh = np.meshgrid(rx_degs, tx_degs, indexing="ij")
beam_results = {"opt": "sdr",
                "taps": si_num_taps,
                "tx": {"degrees": tx_degs.tolist(),
                       "grid": tx_mesh.flatten().tolist()},
                "rx": {"degrees": rx_degs.tolist(),
                       "grid": rx_mesh.flatten().tolist()},
                "tgt_si": [],
                "ach_si": [],
                "spec_nuc_tx": [],
                "spec_nuc_rx": [],
                "beam_dev_tx": [],
                "beam_dev_rx": []
                }
results = []
codebooks = []

phased_array = True
sdrCodebooks = opt.SDRcodebook(h_si, dft_tx_codebook, dft_rx_codebook,
                               si_threshold_power=1.0, beta=0.5, phased=phased_array)
gram_beams_tx = sdrCodebooks.gram_codebook(dft_tx_codebook)
gram_beams_rx = sdrCodebooks.gram_codebook(dft_rx_codebook)


pbar = tqdm(tgt_si_values)
pbar.set_description(f"Baseline SI {si_db_ref:.1f} dB")
for tgt_si_db in pbar:
    
    tgt_si_pow = db2power(tgt_si_db)
    sdrCodebooks.si_threshold_power = tgt_si_pow
        
    pbuf.print(f"Tgt. SI. {tgt_si_db:.1f} dB")

    try:
        start_time = time.perf_counter()
        w_codebook, W_sol = sdrCodebooks.solve(tx=True, return_sdr=True)
        c_codebook, C_sol = sdrCodebooks.solve(tx=False, return_sdr=True)
        end_time = time.perf_counter()
    except RuntimeError as re:
        print("Breaking loop due to runtime issues:")
        print(re)
        pbar.close()
        break

    si_db_sdr = mag2db(opt.sdr_si(W_sol, C_sol, h_si).max())
    loss_w_sdr = pow2db(max(opt.sdr_codebook_deviation(W_sol, gram_beams_tx), 1e-10))
    loss_c_sdr = pow2db(max(opt.sdr_codebook_deviation(C_sol, gram_beams_rx), 1e-10))
    
    max_c_err = opt.maxSdrError(C_sol, c_codebook)
    max_w_err = opt.maxSdrError(W_sol, w_codebook)
    max_amp_err = np.sqrt(max(max_c_err, max_w_err))

    si_db_matrix = cb_si_db(w_codebook, c_codebook, h_si)
    si_db_rank1 = np.max(si_db_matrix)
    tx_dev_beam = np.maximum(opt.codebook_deviation_power(dft_tx_codebook, w_codebook, axis=0), 1e-10)
    rx_dev_beam = np.maximum(opt.codebook_deviation_power(dft_rx_codebook, c_codebook, axis=0), 1e-10)
    loss_w_rank1 = pow2db(np.mean(tx_dev_beam))
    loss_c_rank1 = pow2db(np.mean(rx_dev_beam))

    pbuf.print("Solutions SDR\\Rank-1:")
    pbuf.print(f"\t\tMax. SI.:\t{si_db_sdr:.1f}\\{si_db_rank1:.1f} dB\t({si_db_ref-si_db_rank1:.1f} dB from baseline)")
    pbuf.print(f"\t\tTx error:\t{loss_w_sdr:.1f}\\{loss_w_rank1:.1f} dB")
    pbuf.print(f"\t\tRx error:\t{loss_c_sdr:.1f}\\{loss_c_rank1:.1f} dB")
    pbuf.print(f"\tMax sol. error {max_amp_err:.2E}")
    
    postfix = (f"SI Tgt::Actual {tgt_si_db:.1f}::{si_db_rank1:.1f} dB; "
               f"Error Tx::Rx {loss_w_rank1:.1f}::{loss_c_rank1:.1f} dB; "
               f"Max sol. error {max_amp_err:.2E}")
    pbar.set_postfix_str(postfix)

    if si_num_taps == "full":
        si_db_full = si_db_rank1
    else:
        si_db_full = mag2db(np.max(opt.codebook_si(w_codebook, c_codebook, h_full)))

    if max_amp_err < feasible_tolerance:
        codebooks.append({"si_tgt": tgt_si_db, "si_opt": si_db_rank1, "rx_codebook_p": c_codebook, "tx_codebook_p": w_codebook})
        tx_beam_db = pow2db(tx_dev_beam)
        rx_beam_db = pow2db(rx_dev_beam)
        eps_time = end_time - start_time
    else:
        si_db_rank1 = None
        si_db_full = None
        loss_w_rank1= None
        loss_c_rank1= None
        eps_time = None
        si_db_matrix[:] = np.nan
        tx_beam_db = np.full_like(tx_dev_beam, np.nan)
        rx_beam_db = np.full_like(rx_dev_beam, np.nan)

    beam_results["tgt_si"].append(tgt_si_db)
    beam_results["ach_si"].append(si_db_matrix.flatten().tolist())
    beam_results["spec_nuc_tx"].append(opt.spec_nuc_ratio(W_sol).tolist())
    beam_results["spec_nuc_rx"].append(opt.spec_nuc_ratio(C_sol).tolist())
    beam_results["beam_dev_tx"].append(tx_beam_db.tolist())
    beam_results["beam_dev_rx"].append(rx_beam_db.tolist())

    results.append({"si_tgt": tgt_si_db, "opt": "cissir", "rx_sdr_err": max_c_err, "tx_sdr_err": max_w_err,
                    "si_opt": si_db_rank1, "tx_loss": loss_w_rank1, "rx_loss": loss_c_rank1, "elapsed_time": eps_time,
                    "si_sdr": si_db_sdr, "tx_loss_sdr": loss_w_sdr, "rx_loss_sdr": loss_c_sdr, "si_full": si_db_full,
                    "phased": phased_array, "si_taps": si_num_taps, "num_beams": L_t})

  0%|          | 0/96 [00:00<?, ?it/s]

In [ ]:
opt_times = np.array([r["elapsed_time"] for r in results], dtype=float)
opt_times = opt_times[~np.isnan(opt_times)]
mean_t, median_t = np.mean(opt_times), np.median(opt_times)
print(f"(Mean\Median) Optimization time:\t{mean_t:.2f}\\{median_t:.2f} s")
pbuf.print()

## Saving results

In [ ]:
res_file = res_path/f"cissir_cb_results_phased_{si_num_taps}tap_si.csv"
res_df = pd.DataFrame(results)
res_df.to_csv(res_file)
print(f"Results saved under {res_file}")
json_fname = res_path/f"sdp_beam_results_{si_num_taps}tap.json"
with open(json_fname, 'w', encoding ='utf8') as json_file:
    json.dump(beam_results, json_file)
print(f"Results saved under {json_fname}")

In [ ]:
method = f"phased_cissir{si_num_taps}tap"

si_values = [si_db_ref]
si_achieved = [si_db_ref]
tx_codebooks = [dft_tx_codebook]
rx_codebooks = [dft_rx_codebook]
for cb in codebooks:
    si_values.append(cb["si_tgt"])
    si_achieved.append(cb["si_opt"])
    tx_codebooks.append(cb["tx_codebook_p"])
    rx_codebooks.append(cb["rx_codebook_p"])

si_values = np.array(si_values) 
tx_codebooks = np.stack(tx_codebooks, axis=0)
rx_codebooks = np.stack(rx_codebooks, axis=0)
tx_dev = np.array([-np.inf] + [r["tx_loss"] for r in results]).astype(float)
rx_dev = np.array([-np.inf] + [r["rx_loss"] for r in results]).astype(float)

metadata = np.array({"method": method, "phased": phased_array, "si_num_taps": si_num_taps,
                     "norm": "max"})
fname = f"{method}_codebooks.npz"
np.savez(res_path/fname, si_tgt=si_values,
         si_opt=si_achieved, tx_dev=tx_dev, rx_dev=rx_dev,
         rx_degrees=rx_degs, tx_degrees=tx_degs,
         rx=rx_codebooks, tx=tx_codebooks,
         metadata=metadata)
print(f"{fname} was saved at {res_path}.")

## Plot

In [ ]:
max_grx_db = pow2db(np.max(np.diag(h_si_rx).real))
sat_bnd_db = np.array([np.max(cb_si_db(w_cb, w_cb, h_si_tx)) for w_cb in tx_codebooks])/2 + max_grx_db
sat_opt_db = np.array([np.max(cb_si_db(w_cb, np.eye(N_r), h_si)) for w_cb in tx_codebooks])

plt.plot(si_values, sat_opt_db, label="Achieved SI @Rx-Ant")
plt.plot(si_values, sat_bnd_db, label="Bound SI @Rx-Ant")
plt.legend()
plt.xlim(min(si_values), max(si_values))
plt.ylim(min(sat_opt_db), max(sat_bnd_db))

In [ ]:
with np.load(res_path/"phased_cissir_codebooks.npz") as cb_data:
    si_achieved = cb_data["si_opt"]
    si_values = cb_data["si_tgt"]
    tx_codebooks = cb_data["tx"]
    L_t = tx_codebooks.shape[-1]
    rx_codebooks = cb_data["rx"]
    L_r = rx_codebooks.shape[-1]

    tx_dev = cb_data["tx_dev"]
    rx_dev = cb_data["rx_dev"]

    tx_degs = cb_data["tx_degrees"]
    rx_degs = cb_data["rx_degrees"]
    beam_degs = tx_degs

In [ ]:
plot_si = -100
si_index = np.argmin(np.abs(np.array(si_values) - plot_si))
si_db_val = si_values[si_index]
tx_dev_val = tx_dev[si_index]
opt_tx_codebook = tx_codebooks[si_index,]
opt_rx_codebook = rx_codebooks[si_index,]

In [ ]:
ant_pat = antenna_pattern.v_tr38901_pattern
rad_pattern = lambda x: sum(sum(np.abs(p)**2 for p in ant_pat(np.pi/2, x)))

In [ ]:
f, ax = plt.subplots(subplot_kw={'projection': 'polar'})

rlims = (-9,20)
thetas = np.deg2rad(np.clip(np.arange(-75, 80, 15), -70, 70))
offset, stride = 0, 4

tx_kwargs = dict(transmit=True, element_pattern=rad_pattern, axis=ax, r_lim=rlims, linewidth=0.7, theta_lim=(thetas[0], thetas[-1]))
_ = plot_beamforming_polar(opt_tx_codebook[:, offset::stride], alpha=1.0, color_palette="husl", **tx_kwargs,
                           label="Optimized", zorder=30)
_ = plot_beamforming_polar(dft_tx_codebook[:, offset::stride], alpha=1.0, linestyle="--", color_palette="husl", **tx_kwargs, label="Reference", zorder=40)

#tx_kwargs["linewidth"] = 0.4
#tx_kwargs["alpha"] = 1
_ = plot_beamforming_polar(dft_tx_codebook, aggregate=True, alpha=1.0, linestyle=":", c='k', **tx_kwargs, label="Opt", zorder=10)
#_ = plot_beamforming_polar(opt_tx_codebook, aggregate=True, alpha=1.0, c=[0.93]*3, **tx_kwargs, label="Opt", fill=True, zorder=-20)

bf_lines = ax.get_lines()
for i, d in enumerate(beam_degs[offset::stride]):
    ax.text(np.deg2rad(d), rlims[-1]-1.8, f"{d:.1f}°", c=bf_lines[i].get_color(), rotation=d,
            size="small", rotation_mode="anchor", va="center", ha="center")

plt.rc('axes', axisbelow=False)
#plt.grid(color="gray")

ax.set_xticks(thetas)
xticks = ax.get_xticklabels()
_ = ax.set_xticklabels([tick if i % 2 == 1 else None for i, tick in enumerate(xticks)])
ax.tick_params(axis="x", pad=-1)

handles, labels = ax.get_legend_handles_labels()
ax.legend([handles[i] for i in (0, L_t//stride+1, -1)],
          ["CISSIR", "Reference", 
           "Ref. coverage"], #, "CISSIR coverage"],
          loc="upper center", ncol=3, prop={'size': 9.5},
          columnspacing=0.9,
          bbox_to_anchor=(0.5, 0.98))
ax.set_xlabel("TX array gain [dBi]", labelpad=-30)

bbox = ax.get_tightbbox(bbox_extra_artists=None).transformed(f.dpi_scale_trans.inverted())
bbox_p = bbox.get_points()
bbox_p[0,1] = bbox_p[1,1] - viz.def_figsize[1]*1.05
bbox.set_points(bbox_p)

f.savefig(plot_path/"beams.pdf", bbox_inches=bbox)
f.savefig(plot_path/"beams.pgf", bbox_inches=bbox)